# Minmax Algorithm

In [ ]:
import sys
import requests
from PyQt5.QtWidgets import (
    QApplication,
    QMainWindow,
    QWidget,
    QVBoxLayout,
    QGraphicsDropShadowEffect,
    QPushButton,
    QLineEdit,
    QLabel,
    QInputDialog,
    QStyleFactory,
)
from PyQt5.QtGui import QPainter, QColor, QPen, QFont, QLinearGradient, QIcon, QCursor
from PyQt5.QtCore import Qt, QRectF, QSize, QPointF
import tempfile


class Node:
    def __init__(self, value=None, is_max=True):
        self.value = value
        self.is_max = is_max
        self.children = []


class MinimaxVisualizer(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("Minimax Tree Visualizer")
        self.setGeometry(100, 100, 1000, 800)
        
        # Download the image and save it to a temporary file
        image_url = "https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSUn7FSyBSOYfd4LakAorD8BwBTWxrgobtXCw&s"
        response = requests.get(image_url)
        temp_file = tempfile.NamedTemporaryFile(delete=False, suffix=".jpeg")
        temp_file.write(response.content)
        temp_file.close()
        
        self.setWindowIcon(QIcon(temp_file.name))
        
        self.setStyleSheet(
            """
            QMainWindow {
                background-color: #FFB22C;
            }
            QLabel {
                font-size: 18px;
                font-weight: bold;
                color: #2c2c2c;
            }
            QLineEdit {
                padding: 25px;
                margin-top: 20px;
                border: 4px solid #543310;
                border-radius: 5px;
                font-size: 22px;
                color: #2c2c2c;
                background-color: #F3FEB8;
            }
            QPushButton {
                background-color: #F3FEB8;
                color: #2c2c2c;
                padding: 15px 20px;
                border: none;
                border-radius: 5px;
                font-size: 16px;
                font-weight: bold;
                margin-top: 10px;
                cursor: pointer;
            }
            QPushButton:hover {
                background-color: #FFDE4D;
            }
        """
        )

        main_widget = QWidget()
        main_layout = QVBoxLayout()
        main_layout.setContentsMargins(20, 20, 20, 20)
        main_widget.setLayout(main_layout)
        self.setCentralWidget(main_widget)

        title_label = QLabel("Minimax Tree Visualizer")
        title_label.setAlignment(Qt.AlignCenter)
        title_label.setStyleSheet(
            "font-size: 28px; color: #2c2c2c; font-weight: bold; margin-bottom: 20px;"
        )
        main_layout.addWidget(title_label)

        input_layout = QVBoxLayout()
        self.depth_input = QLineEdit()
        self.depth_input.setPlaceholderText("Enter tree depth")
        self.depth_input.setAlignment(Qt.AlignCenter)
        input_layout.addWidget(QLabel("Tree Depth:"))
        input_layout.addWidget(self.depth_input)

        create_button = QPushButton("Create Tree")
        create_button.clicked.connect(self.create_tree)
        input_layout.addWidget(create_button)
        input_layout.setAlignment(Qt.AlignCenter)

        main_layout.addLayout(input_layout)

        self.canvas = TreeCanvas()
        main_layout.addWidget(self.canvas)

        self.addShadow(create_button)

    def addShadow(self, button):
        button.setGraphicsEffect(
            QGraphicsDropShadowEffect(
                offset=QPointF(5, 5), blurRadius=10, color=QColor(0, 0, 0, 75)
            )
        )
        button.setCursor(QCursor(Qt.PointingHandCursor))

    def create_tree(self):
        try:
            depth = int(self.depth_input.text())
            self.tree = self.create_minimax_tree(depth)
            self.compute_minimax(self.tree)
            self.canvas.tree = self.tree
            self.canvas.set_tree_depth(depth)
            self.canvas.update()
        except ValueError:
            QInputDialog.critical(
                self, "Error", "Please enter a valid integer for tree depth."
            )

    def create_minimax_tree(self, depth, is_max=True):
        if depth == 0:
            value, ok = QInputDialog.getInt(
                self, "Input", "Enter leaf node value:", min=-100, max=100
            )
            if ok:
                return Node(value, is_max)
            return None

        node = Node(is_max=is_max)
        for _ in range(2):  # Binary tree for simplicity
            child = self.create_minimax_tree(depth - 1, not is_max)
            if child:
                node.children.append(child)
            else:
                return None
        return node

    def compute_minimax(self, node):
        if not node.children:
            return node.value

        if node.is_max:
            node.value = max(self.compute_minimax(child) for child in node.children)
        else:
            node.value = min(self.compute_minimax(child) for child in node.children)

        return node.value


class TreeCanvas(QWidget):
    def __init__(self):
        super().__init__()
        self.tree = None
        self.optimal_path = []
        self.depth = 0

    def set_tree_depth(self, depth):
        self.depth = depth
        self.updateGeometry()

    def sizeHint(self):
        width = max(1000, 100 * (2**self.depth))
        height = max(800, 100 * (self.depth + 1))
        return QSize(width, height)

    def paintEvent(self, event):
        if not self.tree:
            return

        painter = QPainter(self)
        painter.setRenderHint(QPainter.Antialiasing)

        def draw_node(node, x, y, width, depth):
            if node.is_max:
                gradient = QLinearGradient(x - 30, y - 30, x + 30, y + 30)
                gradient.setColorAt(0, QColor(52, 152, 219))
                gradient.setColorAt(1, QColor(41, 128, 185))
            else:
                gradient = QLinearGradient(x - 30, y - 30, x + 30, y + 30)
                gradient.setColorAt(0, QColor(231, 76, 60))
                gradient.setColorAt(1, QColor(192, 57, 43))

            painter.setBrush(gradient)

            radius = 30
            rect = QRectF(x - radius, y - radius, 2 * radius, 2 * radius)
            painter.drawEllipse(rect)

            font = QFont("Arial", 14, QFont.Bold)
            painter.setFont(font)
            painter.setPen(Qt.white)
            painter.drawText(rect, Qt.AlignCenter, f"{int(node.value)}")

            if node.children:
                child_width = width / 2
                painter.drawLine(
                    int(x), int(y + radius), int(x - child_width), int(y + 100)
                )
                painter.drawLine(
                    int(x), int(y + radius), int(x + child_width), int(y + 100)
                )

                left_in_path = draw_node(
                    node.children[0], x - child_width, y + 100, child_width, depth + 1
                )
                right_in_path = draw_node(
                    node.children[1], x + child_width, y + 100, child_width, depth + 1
                )

                in_path = left_in_path or right_in_path
                if depth == 0:
                    self.optimal_path = []
                if in_path:
                    self.optimal_path.append((x, y))
                    pen = QPen(QColor(255, 69, 0), 3, Qt.SolidLine)
                    painter.setPen(pen)
                    if left_in_path:
                        painter.drawLine(
                            int(x), int(y + radius), int(x - child_width), int(y + 100)
                        )
                    if right_in_path:
                        painter.drawLine(
                            int(x), int(y + radius), int(x + child_width), int(y + 100)
                        )
                    painter.setPen(Qt.white)

                return in_path
            else:
                return (
                    node.value == self.tree.value
                    if node.is_max
                    else node.value == self.tree.value
                )

        draw_node(self.tree, self.width() / 2, 50, self.width() / 3, 0)

        # Draw optimal path
        if self.optimal_path:
            pen = QPen(QColor(255, 69, 0), 3, Qt.SolidLine)
            painter.setPen(pen)
            for i in range(len(self.optimal_path) - 1):
                x1, y1 = self.optimal_path[i]
                x2, y2 = self.optimal_path[i + 1]
                painter.drawLine(int(x1), int(y1), int(x2), int(y2))


if __name__ == "__main__":
    app = QApplication(sys.argv)
    app.setStyle(QStyleFactory.create("Fusion"))
    window = MinimaxVisualizer()
    window.show()
    sys.exit(app.exec_())


# Code Description

#### This Python code creates a graphical application using PyQt5 to visualize Minimax trees. It includes a main window with a styled interface where users can input the depth of the tree and generate it by clicking a button. The `Node` class represents tree nodes, while the `MinimaxVisualizer` class handles the GUI setup, tree creation, and Minimax value computation. The `TreeCanvas` class is responsible for drawing the tree on the screen, highlighting the optimal path from the root to the optimal leaf node. The application combines user input, recursive tree construction, and graphical rendering to provide an interactive Minimax tree visualization.